In [8]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchsummary import summary
import torchvision
from collections import OrderedDict

nn.Sequential：顺序性 个网络层之间严格按顺序执行，常用于block构建

In [4]:
class LeNetSequential(nn.Module):

    def __init__(self, classes) -> None:
        super(LeNetSequential,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,classes),
        )

    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.classifier(x)

        return x

In [6]:
net = LeNetSequential(2)
summary(net,input_size=(3,32,32),batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [16, 6, 28, 28]             456
              ReLU-2            [16, 6, 28, 28]               0
         MaxPool2d-3            [16, 6, 14, 14]               0
            Conv2d-4           [16, 16, 10, 10]           2,416
              ReLU-5           [16, 16, 10, 10]               0
         MaxPool2d-6             [16, 16, 5, 5]               0
            Linear-7                  [16, 120]          48,120
              ReLU-8                  [16, 120]               0
            Linear-9                   [16, 84]          10,164
             ReLU-10                   [16, 84]               0
           Linear-11                    [16, 2]             170
Total params: 61,326
Trainable params: 61,326
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/ba

In [11]:
class LeNetSequentialOrderDict(nn.Module):

    def __init__(self, classes) -> None:
        super(LeNetSequentialOrderDict,self).__init__()

        self.features = nn.Sequential(OrderedDict({
            'conv1':nn.Conv2d(3,6,5),
            'relu1':nn.ReLU(inplace=True),
            'pool1':nn.MaxPool2d(kernel_size=2,stride=2),

            'conv2': nn.Conv2d(6, 16, 5),
            'relu2': nn.ReLU(inplace=True),
            'pool2': nn.MaxPool2d(kernel_size=2, stride=2),
        }))

        self.classifier = nn.Sequential(OrderedDict({
            'fc1': nn.Linear(16*5*5, 120),
            'relu3': nn.ReLU(),

            'fc2': nn.Linear(120, 84),
            'relu4': nn.ReLU(inplace=True),

            'fc3': nn.Linear(84, classes),
        }))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [12]:
net = LeNetSequentialOrderDict(classes=2)
summary(net,input_size=(3,32,32),batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [16, 6, 28, 28]             456
              ReLU-2            [16, 6, 28, 28]               0
         MaxPool2d-3            [16, 6, 14, 14]               0
            Conv2d-4           [16, 16, 10, 10]           2,416
              ReLU-5           [16, 16, 10, 10]               0
         MaxPool2d-6             [16, 16, 5, 5]               0
            Linear-7                  [16, 120]          48,120
              ReLU-8                  [16, 120]               0
            Linear-9                   [16, 84]          10,164
             ReLU-10                   [16, 84]               0
           Linear-11                    [16, 2]             170
Total params: 61,326
Trainable params: 61,326
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/ba

nn.ModuleList：迭代性，常用于大量重复网络构建 通过for循环实现重复构建

In [13]:
class ModuleList(nn.Module):
    
    def __init__(self) -> None:
        super(ModuleList,self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10,10) for i in range(20)])

    def forward(self,x):
        for i,linear in enumerate(self.linears):
            x = linear(x)

        return x

In [16]:
net = ModuleList()
summary(net,input_size=(4,10),batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [16, 4, 10]             110
            Linear-2                [16, 4, 10]             110
            Linear-3                [16, 4, 10]             110
            Linear-4                [16, 4, 10]             110
            Linear-5                [16, 4, 10]             110
            Linear-6                [16, 4, 10]             110
            Linear-7                [16, 4, 10]             110
            Linear-8                [16, 4, 10]             110
            Linear-9                [16, 4, 10]             110
           Linear-10                [16, 4, 10]             110
           Linear-11                [16, 4, 10]             110
           Linear-12                [16, 4, 10]             110
           Linear-13                [16, 4, 10]             110
           Linear-14                [16

nn.ModuleDict：索引性，常用于可选择的网络层

In [18]:
class ModuleDict(nn.Module):

    def __init__(self) -> None:
        super(ModuleDict,self).__init__()

        self.choices = nn.ModuleDict({
            'conv':nn.Conv2d(10,10,3),
            'pool':nn.MaxPool2d(3),
        })

        self.activations = nn.ModuleDict({
            'relu':nn.ReLU(),
            'prelu':nn.PReLU()
        })

    def forward(self,x,choice,act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x

In [23]:
net = ModuleDict()
fake_img = torch.randn((16,10,32,32))
output = net(fake_img,'conv','relu')
output.shape

torch.Size([16, 10, 30, 30])

AlexNet

In [27]:
alexnet = torchvision.models.AlexNet()
summary(alexnet,input_size=(3,224,224),batch_size=4)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [4, 64, 55, 55]          23,296
              ReLU-2            [4, 64, 55, 55]               0
         MaxPool2d-3            [4, 64, 27, 27]               0
            Conv2d-4           [4, 192, 27, 27]         307,392
              ReLU-5           [4, 192, 27, 27]               0
         MaxPool2d-6           [4, 192, 13, 13]               0
            Conv2d-7           [4, 384, 13, 13]         663,936
              ReLU-8           [4, 384, 13, 13]               0
            Conv2d-9           [4, 256, 13, 13]         884,992
             ReLU-10           [4, 256, 13, 13]               0
           Conv2d-11           [4, 256, 13, 13]         590,080
             ReLU-12           [4, 256, 13, 13]               0
        MaxPool2d-13             [4, 256, 6, 6]               0
AdaptiveAvgPool2d-14             [4, 25